In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyheif

     |████████████████████████████████| 8.2 MB 5.3 MB/s 


In [4]:
!pip install whatimage

In [5]:
!pip install voila

     |████████████████████████████████| 1.6 MB 5.4 MB/s 
     |████████████████████████████████| 551 kB 51.0 MB/s 
     |████████████████████████████████| 393 kB 50.2 MB/s 
     |████████████████████████████████| 112 kB 58.4 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 428 kB 57.6 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 5.6.1
    Uninstalling nbconvert-5.6.1:
      Successfully uninstalled nbconvert-5.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are

In [6]:
from keras.models import load_model, Model
import ipywidgets as widgets
import io
from PIL import Image
import numpy as np
from keras.preprocessing.image import img_to_array
from IPython import display
import whatimage
import pyheif
import tensorflow as tf
import cv2
from keras.preprocessing import image
import keras
import matplotlib.pyplot as plt
import matplotlib.cm as cm


In [7]:
model = load_model('/content/drive/MyDrive/SaffronSystems/Model_checkpoints/fun_model_1.h5')
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
def show_my_image():
    for name, file_info in btn_upload.value.items():
        img = decodeImage((file_info['content'])) 
    #return the image 
    return img.resize((400,400))

In [9]:
def decodeImage(bytesIo):
    fmt = whatimage.identify_image(bytesIo)
    if fmt in ['heic','heif']:
        i = pyheif.read_heif(bytesIo)
        pi = Image.frombytes(mode=i.mode, size=i.size, data=i.data)
    else:
        pi = Image.open(io.BytesIO(bytesIo))
    return pi

In [10]:
def get_img_array(img, size):
    # `img` is a PIL image of size 299x299
    img = img.resize(size=size)
    # `array` is a float32 Numpy array of shape (224, 224, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 224, 224, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer 
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [11]:
def save_and_display_gradcam(img, heatmap, alpha=0.4):

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    return superimposed_img

In [21]:
def predict(image: Image.Image,alpha=0.4):
    for name, file_info in btn_upload.value.items():
        img = decodeImage((file_info['content']))
    orignal_image = img_to_array(img.resize(size=(224,224)))
    img = np.expand_dims(orignal_image, axis = 0)
    img = img / 127.5 - 1.0    
    result = model.predict(img)
    print(result)
    if result[0][0] > 0.50:
      lbl_pred.value = "Predicted Class : " + "Saffron" 
    else:
      lbl_pred.value = "Predicted Class : " + "Non-Saffron"
    saffron_prob.value = "probability of Saffron : " +str(round(result[0][0]*100,2))+"%"
    nsaffron_prob.value = "probability of Non-Saffron: " +str(round((1-result[0][0])*100,2))+"%"
    out_pl.clear_output()
    out_heatmap.clear_output()
     # display the uploaded image
    # Remove last layer's softmax
    vis_model = model
    vis_model.layers[-1].activation = None

    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(img, vis_model, 'block5_conv3')
    sup_img = save_and_display_gradcam(orignal_image,heatmap)
    with out_pl:
        display.display(show_my_image())
    with out_heatmap:
        display.display(sup_img)

In [22]:
# Elements Of the App
# upload Button
btn_upload = widgets.FileUpload()
# output widget to display image 
out_pl = widgets.Output()
# output widget to display image 
out_heatmap = widgets.Output()
# label for the result of prediction
lbl_pred = widgets.Label()
saffron_prob = widgets.Label()
nsaffron_prob = widgets.Label()
# Button to classify the uploaded image 
btn_run = widgets.Button(description='Classify')
# event for classify button to predict the result and display uploaded image
image = btn_run.on_click(predict)

In [23]:
widgets.VBox([widgets.Label('Select Image'),btn_upload,btn_run,out_pl,out_heatmap,lbl_pred,saffron_prob,nsaffron_prob],box_style='info')

VBox(box_style='info', children=(Label(value='Select Image'), FileUpload(value={}, description='Upload'), Butt…

[[0.87611645]]
[[0.0004856]]
[[4.4104076e-05]]
